In [1]:
#!pip install Levenshtein

import sys
import pandas as pd
import math
import numpy as np
import logging

# sys.path.append("C:/Users/sina_/Bachelor_Arbeit/Src/Fine-Tuning")
sys.path.append("C:/Users/sina_/Bachelor_Arbeit/Src/Feature_Extractions")
sys.path.append("C:/Users/sina_/Bachelor_Arbeit/Src/Housnumber")
sys.path.append("C:/Users/sina_/Bachelor_Arbeit/Src/Blocker")
sys.path.append("C:/Users/sina_/Bachelor_Arbeit/Src/Matcher")
sys.path.append("C:/Users/sina_/Bachelor_Arbeit/Src/Fine-Tuning-new")


from fine_tuning3 import *
from Housnumber import *
from Blocker import *
from feature_extraction import *
from Matcher import *

logger = logging.getLogger()
import warnings

warnings.filterwarnings(
    "ignore",
    category=FutureWarning,
)
pd.options.mode.chained_assignment = None

C:\Users\sina_\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# nvidia-smi
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [57]:
pd.set_option("display.max_rows", 20)
pd.set_option("display.max_colwidth", None)

# Hugging  Face Daten

In [4]:
df_hugging = pd.read_parquet(
    "C:/Users/sina_/Bachelor_Arbeit/Src/Data&Labels/Data/hugging_face_clean.parquet"
)
df_hugging

,COMPANYNAME,STREET,POSTALCODE,CITY,HSNR
2,Bruno Söhnle GmbH Uhrenatelier Glashütte/Sa.,Dresdner Str.,01768,Glashütte,14
3,Funke & Will AG,Zum Fliegerhorst,01558,Großenhain,11
4,f6 Cigarettenfabrik GmbH & Co. KG,Junghansstr.,01277,Dresden,5
5,BASF Schwarzheide GmbH,Schipkauer Str.,01987,Schwarzheide,1
6,Elbe Flugzeugwerke GmbH,Grenzstr.,01109,Dresden,1
...,...,...,...,...,...
261867,gert feistkorn,Haunoldshofener Weg,90599,Dietenhofen,3
261868,christian hans,Schweppermannstr.,90408,Nürnberg,23
261869,peter fürderer,An der Fleischbrücke,90455,Nürnberg,1-3
261870,peter kühnl,Habsburgerstr.,90475,Nürnberg,24


## OSM Daten

In [5]:
df_osm = pd.read_parquet(
    "C:/Users/sina_/Bachelor_Arbeit/Src/Data&Labels/Data/df_osm_all.parquet"
)
df_osm = df_osm[["COMPANYNAME", "CITY", "POSTALCODE", "STREET", "HSNR"]]
df_osm

,COMPANYNAME,CITY,POSTALCODE,STREET,HSNR
0,BBBank,Weinheim,69469,Dürrestraße,1
1,Arbeitsgericht Karlsruhe,Karlsruhe,76133,Ritterstraße,12
2,Sparkasse,Karlsruhe,76199,Tulpenstraße,4
3,Marienapotheke,Karlsruhe,76137,Marienstraße,43
4,Spitzwegapotheke,Karlsruhe,76137,Südendstraße,29
...,...,...,...,...,...
401316,Gemeinde Herbsleben,Herbsleben,99955,Mitteltor,4
401317,Johanniter-Unfall-Hilfe e.V.,Eisenberg,07607,Klosterlausnitzer Straße,23
401318,Gemeindeverwaltung,Frankenblick,96528,Alter Weg,24
401319,Anja Henkel,Suhl,98527,Albert-Schweitzer-Straße,1


In [72]:
# viele von Osm Postleitzahlen sind nicht in hugging face  drin
df_hugging = df_hugging[df_hugging["POSTALCODE"].isin(df_osm["POSTALCODE"])]
df_osm = df_osm[df_osm["POSTALCODE"].isin(df_hugging["POSTALCODE"])]
df_osm

,COMPANYNAME,CITY,POSTALCODE,STREET,HSNR
1,Arbeitsgericht Karlsruhe,Karlsruhe,76133,Ritterstraße,12
2,Sparkasse,Karlsruhe,76199,Tulpenstraße,4
3,Marienapotheke,Karlsruhe,76137,Marienstraße,43
4,Spitzwegapotheke,Karlsruhe,76137,Südendstraße,29
23,Oberwaldschule Aue,Karlsruhe,76227,Grazer Straße,25
...,...,...,...,...,...
401306,Friedrich-Schiller-Universität Jena,Jena,07743,Ernst-Abbe-Platz,2
401308,Johanniter-Unfall-Hilfe e.V.,Uhlstädt-Kirchhasel,07407,Engerda,17
401309,Gemeinde Christes,Christes,98547,Metzelser Straße,20
401315,Jugendsozialwerk Rudolstadt gGmbH,Rudolstadt,07407,Pestalozzistraße,11


In [73]:
blocked_df = plz_blocking(df_osm, df_hugging, "POSTALCODE")
# blocked_df

In [75]:
all_candidates = pd.DataFrame()

for block in blocked_df.values():
    all_candidates = pd.concat([all_candidates, Indexing(block)], ignore_index=True)

all_candidates

,COMPANYNAME_DF1,POSTALCODE_DF1,CITY_DF1,STREET_DF1,HSNR_DF1,INDEX_SOURCE_DF1,INDEX_SOURCE_DF2,COMPANYNAME_DF2,POSTALCODE_DF2,CITY_DF2,STREET_DF2,HSNR_DF2
0,BBBank,01067,Dresden,Altmarkt,10,89993,143717,Interessenverbund sächsischer Mehrgenerationenhäuser e.V.,01067,Dresden,Adlergasse,14
1,alfatraining Bildungszentrum,01067,Dresden,Altmarkt,10a,90498,143717,Interessenverbund sächsischer Mehrgenerationenhäuser e.V.,01067,Dresden,Adlergasse,14
2,AllDent Zahnzentrum Dresden GmbH,01067,Dresden,Altmarkt,10a-d,128078,143717,Interessenverbund sächsischer Mehrgenerationenhäuser e.V.,01067,Dresden,Adlergasse,14
3,Zahnzentrum AllDent,01067,Dresden,Altmarkt,10a-d,90655,143717,Interessenverbund sächsischer Mehrgenerationenhäuser e.V.,01067,Dresden,Adlergasse,14
4,Vögele,01067,Dresden,Altmarkt,21,89808,143717,Interessenverbund sächsischer Mehrgenerationenhäuser e.V.,01067,Dresden,Adlergasse,14
...,...,...,...,...,...,...,...,...,...,...,...,...
1219401,Hartmann Naturbaustoffe,99820,Hörselberg-Hainich,Mühlweg,40,104113,183225,IWESA Präzisionsteile GmbH,99820,Hörselberg-Hainich,Industriestr.,4
1219402,Salon Hauptsache,99820,Hörselberg-Hainich,Neue Straße,92,103992,183225,IWESA Präzisionsteile GmbH,99820,Hörselberg-Hainich,Industriestr.,4
1219403,Dipl. med. Kirsch,99820,Hörselberg-Hainich,Neue Straße,92a,102785,183225,IWESA Präzisionsteile GmbH,99820,Hörselberg-Hainich,Industriestr.,4
1219404,Regenbogen,99820,Hörselberg-Hainich,Sottengarten,1,104844,183225,IWESA Präzisionsteile GmbH,99820,Hörselberg-Hainich,Industriestr.,4


In [ ]:
temp1 = df_osm["POSTALCODE"].value_counts().reset_index()
temp2 = df_hugging["POSTALCODE"].value_counts().reset_index()
temp3 = temp1.merge(temp2, on="POSTALCODE", suffixes=("_OSM", "_HUG"))
temp3["SUM"] = temp3["count_OSM"] + temp3["count_HUG"]
temp3["window_size"] = temp3.apply(
    lambda row: window(row["count_OSM"], row["count_HUG"]), axis=1
)
temp3.sort_values("SUM", ascending=False)

In [ ]:
temp4 = all_candidates["POSTALCODE_DF1"].value_counts().reset_index()
all_candidates_plz = temp3.merge(temp4, left_on="POSTALCODE", right_on="POSTALCODE_DF1")
all_candidates_plz

In [ ]:
a = df_osm[~df_osm["COMPANYNAME"].isin(all_candidates["COMPANYNAME_DF1"])]
b = df_hugging[~df_hugging["COMPANYNAME"].isin(all_candidates["COMPANYNAME_DF2"])]
nicht_enthalten = pd.merge(a, b, on=["COMPANYNAME", "POSTALCODE"], how="inner")

nicht_enthalten

In [ ]:
pd.merge(a, b, on=["COMPANYNAME"], how="inner")

In [76]:
can_hsnr = hsnr_preprocessing(all_candidates, "HSNR_DF1")
can_hsnr = hsnr_preprocessing(all_candidates, "HSNR_DF2")
can_hsnr = compare_house_numbers(
    can_hsnr, "HSNR_DF1_CLEAN", "HSNR_DF2_CLEAN", threshold=0.80
)

can_hsnr

,COMPANYNAME_DF1,POSTALCODE_DF1,CITY_DF1,STREET_DF1,HSNR_DF1,INDEX_SOURCE_DF1,INDEX_SOURCE_DF2,COMPANYNAME_DF2,POSTALCODE_DF2,CITY_DF2,STREET_DF2,HSNR_DF2,HSNR_SIM
36,BBBank,01067,Dresden,Altmarkt,10,89993,136834,Cotton House Vermögensverwaltung GmbH,01067,Dresden,Altmarkt,10,1.0
37,BBBank,01067,Dresden,Altmarkt,10,89993,135904,Dresden Altmarkt BC GP GmbH,01067,Dresden,Altmarkt,10,1.0
38,BBBank,01067,Dresden,Altmarkt,10,89993,140543,Emica-Bau GmbH,01067,Dresden,Altmarkt,10,1.0
39,BBBank,01067,Dresden,Altmarkt,10,89993,135545,Grundbesitz Lauensteinerstr. GmbH & Co. KG,01067,Dresden,Altmarkt,10,1.0
40,BBBank,01067,Dresden,Altmarkt,10,89993,136756,Hp Elektro-Projekt UG (haftungsbeschränkt),01067,Dresden,Altmarkt,10,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219276,Private Fachschule für Wirtschaft und Soziales gGmbH,99099,Erfurt,Sorbenweg,4,132722,256021,Impuls TV GmbH & Co. KG,99099,Erfurt,Krohstr.,4,1.0
1219287,HIMA GmbH Niederlassung Berlstedt,99439,Am Ettersberg,Industriegebiet,1,103657,286696,STW GmbH,99439,Ramsla,Heichelheimer Weg,1,1.0
1219298,Lyonel-Feininger-Gymnasium,99439,Am Ettersberg,Volkmarsener Platz,1,103623,286696,STW GmbH,99439,Ramsla,Heichelheimer Weg,1,1.0
1219340,Fahrschule Leischner,99817,Eisenach,Unterstraße,3,104105,292756,Stitz GmbH,99817,Eisenach,Zeppelinstr.,3,1.0


In [79]:
df1_predicted = load_model_and_predict(
    "cuda",
    "C:/Users/sina_/Bachelor_Arbeit/Src/Fine-Tuning-new/OSM/Best_Model_CrossValidation/Street/fold_4/model",
    df=can_hsnr,
    batch_size=16,
    pred_street=True,
    pred_company=False,
)
df1_predicted
# df2_predicted  =load_model_and_predict('cuda' , company_model_path , df1_predicted ,batch_size = batch_size, pred_street= False , pred_company = True)

,COMPANYNAME_DF1,POSTALCODE_DF1,CITY_DF1,STREET_DF1,HSNR_DF1,INDEX_SOURCE_DF1,INDEX_SOURCE_DF2,COMPANYNAME_DF2,POSTALCODE_DF2,CITY_DF2,STREET_DF2,HSNR_DF2,HSNR_SIM,STREET_SERIALIZED,PREDICTIONS_STREET
36,BBBank,01067,Dresden,Altmarkt,10,89993,136834,Cotton House Vermögensverwaltung GmbH,01067,Dresden,Altmarkt,10,1.0,Altmarkt [SEP] Altmarkt,1
37,BBBank,01067,Dresden,Altmarkt,10,89993,135904,Dresden Altmarkt BC GP GmbH,01067,Dresden,Altmarkt,10,1.0,Altmarkt [SEP] Altmarkt,1
38,BBBank,01067,Dresden,Altmarkt,10,89993,140543,Emica-Bau GmbH,01067,Dresden,Altmarkt,10,1.0,Altmarkt [SEP] Altmarkt,1
39,BBBank,01067,Dresden,Altmarkt,10,89993,135545,Grundbesitz Lauensteinerstr. GmbH & Co. KG,01067,Dresden,Altmarkt,10,1.0,Altmarkt [SEP] Altmarkt,1
40,BBBank,01067,Dresden,Altmarkt,10,89993,136756,Hp Elektro-Projekt UG (haftungsbeschränkt),01067,Dresden,Altmarkt,10,1.0,Altmarkt [SEP] Altmarkt,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219276,Private Fachschule für Wirtschaft und Soziales gGmbH,99099,Erfurt,Sorbenweg,4,132722,256021,Impuls TV GmbH & Co. KG,99099,Erfurt,Krohstr.,4,1.0,Sorbenweg [SEP] Krohstr.,0
1219287,HIMA GmbH Niederlassung Berlstedt,99439,Am Ettersberg,Industriegebiet,1,103657,286696,STW GmbH,99439,Ramsla,Heichelheimer Weg,1,1.0,Industriegebiet [SEP] Heichelheimer Weg,0
1219298,Lyonel-Feininger-Gymnasium,99439,Am Ettersberg,Volkmarsener Platz,1,103623,286696,STW GmbH,99439,Ramsla,Heichelheimer Weg,1,1.0,Volkmarsener Platz [SEP] Heichelheimer Weg,0
1219340,Fahrschule Leischner,99817,Eisenach,Unterstraße,3,104105,292756,Stitz GmbH,99817,Eisenach,Zeppelinstr.,3,1.0,Unterstraße [SEP] Zeppelinstr.,0


In [80]:
df1_predicted[df1_predicted["PREDICTIONS_STREET"] == 1]

,COMPANYNAME_DF1,POSTALCODE_DF1,CITY_DF1,STREET_DF1,HSNR_DF1,INDEX_SOURCE_DF1,INDEX_SOURCE_DF2,COMPANYNAME_DF2,POSTALCODE_DF2,CITY_DF2,STREET_DF2,HSNR_DF2,HSNR_SIM,STREET_SERIALIZED,PREDICTIONS_STREET
36,BBBank,01067,Dresden,Altmarkt,10,89993,136834,Cotton House Vermögensverwaltung GmbH,01067,Dresden,Altmarkt,10,1.0,Altmarkt [SEP] Altmarkt,1
37,BBBank,01067,Dresden,Altmarkt,10,89993,135904,Dresden Altmarkt BC GP GmbH,01067,Dresden,Altmarkt,10,1.0,Altmarkt [SEP] Altmarkt,1
38,BBBank,01067,Dresden,Altmarkt,10,89993,140543,Emica-Bau GmbH,01067,Dresden,Altmarkt,10,1.0,Altmarkt [SEP] Altmarkt,1
39,BBBank,01067,Dresden,Altmarkt,10,89993,135545,Grundbesitz Lauensteinerstr. GmbH & Co. KG,01067,Dresden,Altmarkt,10,1.0,Altmarkt [SEP] Altmarkt,1
40,BBBank,01067,Dresden,Altmarkt,10,89993,136756,Hp Elektro-Projekt UG (haftungsbeschränkt),01067,Dresden,Altmarkt,10,1.0,Altmarkt [SEP] Altmarkt,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1217509,"Autohaus,Land-und Gartentechnik Milde",92274,Gebenbach,Am Weingarten,5,28877,212728,MILDE GmbH,92274,Gebenbach,Am Weingarten,5,1.0,Am Weingarten [SEP] Am Weingarten,1
1217995,SF Gebäudereinigung,95030,Hof,Hofer Straße,21,27772,332126,SF Gebäudereinigung GmbH,95030,Hof,Hofer Str.,21,1.0,Hofer Straße [SEP] Hofer Str.,1
1218666,CITYCOM GmbH,97239,Aub,Industriestraße,5-9,16236,294641,CITYCOM GmbH,97239,Aub,Industriestr.,5-9,1.0,Industriestraße [SEP] Industriestr.,1
1218788,Reich GmbH,97638,Mellrichstadt,Industriestraße,1,19221,183925,Reich GmbH,97638,Mellrichstadt,Industriestr.,1,1.0,Industriestraße [SEP] Industriestr.,1


In [ ]:
can_hsnr["HSNR_SIM"].value_counts()

In [ ]:
filtered_df_hsnr1 = can_hsnr[
    (can_hsnr["HSNR_SIM"] >= 0.76)
    & (
        can_hsnr["HSNR_DF1"]
        .astype(str)
        .str.contains("[a-zA-ZäÄöÖüÜß]{1,3}", regex=True)
    )
][["HSNR_DF1", "HSNR_DF2", "HSNR_SIM"]]

filtered_df_hsnr1 = filtered_df_hsnr1.drop_duplicates(subset=["HSNR_SIM", "HSNR_DF1"])


filtered_df_hsnr1 = filtered_df_hsnr1.sort_values("HSNR_SIM", ascending=False)


result1 = filtered_df_hsnr1.groupby("HSNR_SIM").head(5)

result1[["HSNR_DF1", "HSNR_DF2", "HSNR_SIM"]].sort_values(
    "HSNR_SIM", ascending=False
).head(20)

In [ ]:
filtered_df_hsnr = can_hsnr[(can_hsnr["HSNR_SIM"] >= 0.48)]

filtered_df_hsnr = filtered_df_hsnr.drop_duplicates(subset=["HSNR_SIM", "HSNR_DF1"])


filtered_df_hsnr = filtered_df_hsnr.sort_values("HSNR_SIM", ascending=False)


result = filtered_df_hsnr.groupby("HSNR_SIM").head(5)

result[["HSNR_DF1", "HSNR_DF2", "HSNR_SIM"]].sort_values(
    "HSNR_SIM", ascending=False
).head(20)

In [ ]:
can_hsnr[(can_hsnr["HSNR_SIM"] >= 0.80) & (can_hsnr["HSNR_SIM"] != 1)][
    ["HSNR_DF1", "HSNR_DF2", "HSNR_SIM"]
].drop_duplicates(subset={"HSNR_SIM", "HSNR_DF1"}).sort_values(
    "HSNR_SIM", ascending=False
).tail(
    10
)

### Anwendung des Fine-Tuned Model


In [8]:
df_osm["COMPANYNAME_ORG"] = df_osm["COMPANYNAME"].copy()
df_hugging["COMPANYNAME_ORG"] = df_hugging["COMPANYNAME"].copy()

df_osm["COMPANYNAME"] = df_osm["COMPANYNAME"].str.lower()
df_hugging["COMPANYNAME"] = df_hugging["COMPANYNAME"].str.lower()

In [6]:
matched_lower = matcher(
    df_osm,
    df_hugging,
    batch_size=16,
    company_model_path="C:/Users/sina_/Bachelor_Arbeit/Src/Fine-Tuning-new/OSM/Company_Model_16_batchsize/model",
)
matched_lower

,COMPANYNAME_DF1,POSTALCODE_DF1,CITY_DF1,STREET_DF1,HSNR_DF1,INDEX_SOURCE_DF1,INDEX_SOURCE_DF2,COMPANYNAME_DF2,POSTALCODE_DF2,CITY_DF2,STREET_DF2,HSNR_DF2,HSNR_SIM,PREDICTIONS_STREET,PREDICTIONS_COMPANY,MATCH_ID
692,Dresden International School,01067,Dresden,Annenstraße,9,98090,133909,DIS Dresden International School gemeinnützige GmbH,01067,Dresden,Annenstr.,9,1.0,1,1,1
697,Dresden International School,01067,Dresden,Annenstraße,9,98090,143899,Japanische Ergänzungsschule in Dresden e.V.,01067,Dresden,Annenstr.,9,1.0,1,1,1
1628,"Dr. med. Antje Heilmann, Dr. med. Helge Tomczak",01067,Dresden,Friedrichstraße,38,128064,144377,Kinderzentrum Dresden-Friedrichstadt GmbH,01067,Dresden,Friedrichstr.,38,1.0,1,1,2
1630,Praxis für Kinderkardiologie,01067,Dresden,Friedrichstraße,38,90608,144377,Kinderzentrum Dresden-Friedrichstadt GmbH,01067,Dresden,Friedrichstr.,38,1.0,1,1,2
1644,Praxis für Humangenetik,01067,Dresden,Friedrichstraße,38-40,94753,136812,Medizinisches Versorgungszentrum Mitteldeutscher Praxisverbund Humangenetik GmbH,01067,Dresden,Friedrichstr.,38-40,1.0,1,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1215878,Apotheke zur Grünen Schlange,90768,Fürth,Kapellenplatz,1,15293,336076,Apotheke Zur grünen Schlange Johannes Kimberger e. K.,90768,Fürth,Kapellenplatz,1,1.0,1,1,2202
1216593,Frosch-Apotheke,90768,Fürth,Vacher Straße,462,17145,334619,Frosch - Apotheke e. K.,90768,Fürth,Vacher Str.,462,1.0,1,1,2203
1217995,SF Gebäudereinigung,95030,Hof,Hofer Straße,21,27772,332126,SF Gebäudereinigung GmbH,95030,Hof,Hofer Str.,21,1.0,1,1,2204
1218666,CITYCOM GmbH,97239,Aub,Industriestraße,5-9,16236,294641,CITYCOM GmbH,97239,Aub,Industriestr.,5-9,1.0,1,1,2205


In [ ]:
matched_lower.columns

In [7]:
# 16 bit 5 epoches 2444
# best fold auch mit 16
# ohne klein buchstaben
matched_lower["MATCH_ID"].nunique()

2206

In [ ]:
matched_lower.to_parquet("matched_lower.parquet")

In [ ]:
matched.to_parquet("matched.parquet")

In [ ]:
matched[
    [
        "MATCH_ID",
        "COMPANYNAME_DF1",
        "COMPANYNAME_DF2",
        "STREET_DF1",
        "STREET_DF2",
        "HSNR_DF1",
        "HSNR_DF2",
    ]
].sample(10)

In [ ]:
street_best_fold_model_path = "C:/Users/sina_/Bachelor_Arbeit/Src/Fine-Tuning-new/OSM/Best_Model_CrossValidation/Street/fold_4/model"
company_best_fold_model_path = "C:/Users/sina_/Bachelor_Arbeit/Src/Fine-Tuning-new/OSM/Best_Model_CrossValidation/Company/fold_4/model"

In [ ]:
matched[
    (matched["PREDICTIONS_COMPANY"] == 1)
    & (matched["PREDICTIONS_STREET"] == 1)
    & (matched["COMPANYNAME_DF1"] != matched["COMPANYNAME_DF2"])
].drop_duplicates(["COMPANYNAME_DF1", "COMPANYNAME_DF2"])[
    ["COMPANYNAME_DF1", "COMPANYNAME_DF2", "STREET_DF1", "STREET_DF2"]
]

# Vergleich Matrix matcher vs Fine-Tuning

In [8]:
vergleich_mm_ft = pd.read_parquet("match_vergleich.parquet")
vergleich_mm_ft["_merge"] = vergleich_mm_ft["_merge"].replace(
    {"right_only": "MatrixMatcher", "both": "both", "left_only": "Fine-Tuning"}
)
vergleich_mm_ft

,COMPANYNAME_OSM,POSTALCODE_OSM_x,CITY_OSM_x,STREET_OSM_x,HSNR_OSM_x,INDEX_SOURCE_OSM,INDEX_SOURCE_HUG,COMPANYNAME_HUG,POSTALCODE_HUG_x,CITY_HUG_x,...,POSTALCODE_HUG_y,CITY_HUG_y,HSNR_HUG_y,COMPANYNAME_worddiff,STREET_worddiff_locality,HSNR_housenumber,POSTALCODE_identity,CITY_worddiff_locality,CASE,_merge
0,(smow),10709,Berlin,Kurfürstendamm,100,33028,149812,smow B GmbH,10709,Berlin,...,10709,Berlin,100,0.884587,1.0,1.0,1.0,1.0,case_0,both
1,107. Oberschule + 108. Grundschule „Sonnenblumenschule“,01309,Dresden,Hepkestraße,26-28,97371,141071,Förderverein der 107. Oberschule Dresden e.V.,01309,Dresden,...,None,None,None,NaN,NaN,NaN,NaN,NaN,None,Fine-Tuning
2,1concept,10715,Berlin,Detmolder Straße,66,34591,171030,1concept Makler GmbH,10715,Berlin,...,None,None,None,NaN,NaN,NaN,NaN,NaN,None,Fine-Tuning
3,3D Personal,10713,Berlin,Brandenburgische Straße,78/79,34686,155728,3D Personal e.K.,10713,Berlin,...,None,None,None,NaN,NaN,NaN,NaN,NaN,None,Fine-Tuning
4,471 Service,90471,Nürnberg,Kafkastraße,6,14123,339409,471 Service GmbH,90471,Nürnberg,...,90471,Nürnberg,6,1.000000,1.0,1.0,1.0,1.0,case_0,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3842,xapio GmbH,80636,München,Nymphenburger Straße,90e,20381,322679,xapio GmbH,80636,München,...,None,None,None,NaN,NaN,NaN,NaN,NaN,None,Fine-Tuning
3843,xit GmbH,None,None,None,None,None,None,XIT Data GmbH,None,None,...,90443,Nürnberg,73,0.884587,1.0,1.0,1.0,1.0,case_0,MatrixMatcher
3844,Ärztehaus Dresden-Blasewitz,01309,Dresden,Naumannstraße,3,97938,144034,Kniezentrum Dresden Dr. med. J.-H. Fischer und A. Nikulski Partnerschaft von Ärzten und Physiotherapeuten,01309,Dresden,...,None,None,None,NaN,NaN,NaN,NaN,NaN,None,Fine-Tuning
3845,Ärztehaus Fellbach,70734,Fellbach,Bahnhofstraße,52,9029,297621,Stadt-Apotheke Fellbach e.K.,70734,Fellbach,...,None,None,None,NaN,NaN,NaN,NaN,NaN,None,Fine-Tuning


In [9]:
vergleich_mm_ft["_merge"].value_counts()

_merge
Fine-Tuning      1717
both             1467
MatrixMatcher     663
Name: count, dtype: int64

In [10]:
from Levenshtein import ratio


def levenstein_rario(str1, str2):
    dist = ratio(str1, str2)
    return dist

In [ ]:
nur_ft = vergleich_mm_ft[vergleich_mm_ft["_merge"] == "Fine-Tuning"][
    [
        "COMPANYNAME_OSM",
        "POSTALCODE_OSM_x",
        "CITY_OSM_x",
        "STREET_OSM_x",
        "HSNR_OSM_x",
        "INDEX_SOURCE_OSM",
        "INDEX_SOURCE_HUG",
        "COMPANYNAME_HUG",
        "POSTALCODE_HUG_x",
        "CITY_HUG_x",
        "STREET_HUG_x",
        "HSNR_HUG_x",
        "HSNR_SIM",
        "PREDICTIONS_STREET",
        "PREDICTIONS_COMPANY",
        "MATCH_ID",
        "LEV_DIST",
    ]
]
nur_ft.drop_duplicates(
    subset={"COMPANYNAME_OSM", "COMPANYNAME_HUG", "MATCH_ID"}, inplace=True
)
nur_ft

In [ ]:
nur_ft["LEV_DIST"].astype(str).str[:3].value_counts()

In [ ]:
nur_ft[nur_ft["LEV_DIST"] < 0.35].sample(10)

In [ ]:
nur_MM = vergleich_mm_ft[vergleich_mm_ft["_merge"] == "MatrixMatcher"][
    [
        "COMPANYNAME_OSM",
        "POSTALCODE_OSM_y",
        "CITY_OSM_y",
        "STREET_OSM_y",
        "HSNR_OSM_y",
        "COMPANYNAME_HUG",
        "POSTALCODE_HUG_y",
        "CITY_HUG_y",
        "STREET_HUG_y",
        "HSNR_HUG_y",
        "COMPANYNAME_worddiff",
        "STREET_worddiff_locality",
        "HSNR_housenumber",
        "POSTALCODE_identity",
        "CITY_worddiff_locality",
        "match_ID",
    ]
]
nur_MM

In [ ]:
nur_MM[
    nur_MM.duplicated(
        subset={"COMPANYNAME_OSM", "COMPANYNAME_HUG", "match_ID"}, keep=False
    )
].head(10)

In [ ]:
vergleich_mm_ft.columns

In [38]:
mm = vergleich_mm_ft[vergleich_mm_ft["_merge"].isin({"MatrixMatcher", "both"})][
    [
        "COMPANYNAME_OSM",
        "COMPANYNAME_HUG",
        "CITY_OSM_y",
        "POSTALCODE_OSM_y",
        "STREET_OSM_y",
        "HSNR_OSM_y",
        "STATE",
        "match_ID",
        "STREET_HUG_y",
        "POSTALCODE_HUG_y",
        "CITY_HUG_y",
        "HSNR_HUG_y",
        "COMPANYNAME_worddiff",
        "STREET_worddiff_locality",
        "HSNR_housenumber",
        "POSTALCODE_identity",
        "CITY_worddiff_locality",
        "CASE",
    ]
]
mm.columns = mm.columns.str.replace("_y", "")
mm

,COMPANYNAME_OSM,COMPANYNAME_HUG,CITY_OSM,POSTALCODE_OSM,STREET_OSM,HSNR_OSM,STATE,match_ID,STREET_HUG,POSTALCODE_HUG,CITY_HUG,HSNR_HUG,COMPANYNAME_worddiff,STREET_worddiff_locality,HSNR_housenumber,POSTALCODE_identity,CITY_worddiff_locality,CASE
0,(smow),smow B GmbH,Berlin,10709,Kurfürstendamm,100,Berlin,184,Kurfürstendamm,10709,Berlin,100,0.884587,1.0,1.0,1.0,1.0,case_0
4,471 Service,471 Service GmbH,Nürnberg,90471,Kafkastraße,6,Bayern,1521,Kafkastr.,90471,Nürnberg,6,1.000000,1.0,1.0,1.0,1.0,case_0
7,500elf Autoservice,500elf Autoservice GmbH,Köln,50968,Bonner Straße,511,Nordrhein-Westfalen,1168,Bonner Str.,50968,Köln,511,1.000000,1.0,1.0,1.0,1.0,case_0
8,A&A Digitalprint,A & A Digitalprint GmbH,Düsseldorf,40210,Karlstraße,31,Nordrhein-Westfalen,441,Karlstr.,40210,Düsseldorf,31,1.000000,1.0,1.0,1.0,1.0,case_0
13,A2 Center,"Apotheke im A2 Center, e.K.",Isernhagen,30916,Opelstraße,3-5,Niedersachsen,1017,Opelstr.,30916,Isernhagen,3-5,0.884587,1.0,1.0,1.0,1.0,case_0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3829,ver.di Service GmbH,ver.di Service GmbH,Berlin,10243,Franz-Mehring-Platz,1,Berlin,72,Franz-Mehring-Platz,10243,Berlin,1,1.000000,1.0,1.0,1.0,1.0,case_0
3833,visionate interactive interiors GmbH,visionate interactive interiors GmbH,Hannover,30459,Göttinger Chaussee,115,Niedersachsen,944,Göttinger Chaussee,30459,Hannover,115,1.000000,1.0,1.0,1.0,1.0,case_0
3835,wagnis eG,Wohnbaugenossenschaft wagnis eG,München,80797,Petra-Kelly-Straße,29,Bayern,1372,Petra-Kelly-Str.,80797,München,29,0.884587,1.0,1.0,1.0,1.0,case_0
3836,werk21 GmbH,werk21 GmbH,Berlin,10115,Krausnickstraße,3,Berlin,46,Krausnickstr.,10115,Berlin,3,1.000000,1.0,1.0,1.0,1.0,case_0


In [12]:
# matched_lower = pd.read_parquet("matched_lower.parquet")
matched_lower.columns = matched_lower.columns.str.replace("_DF1", "_OSM").str.replace(
    "_DF2", "_HUG"
)
matched_lower  # .columns

,COMPANYNAME_OSM,POSTALCODE_OSM,CITY_OSM,STREET_OSM,HSNR_OSM,INDEX_SOURCE_OSM,INDEX_SOURCE_HUG,COMPANYNAME_HUG,POSTALCODE_HUG,CITY_HUG,STREET_HUG,HSNR_HUG,HSNR_SIM,PREDICTIONS_STREET,PREDICTIONS_COMPANY,MATCH_ID
692,Dresden International School,01067,Dresden,Annenstraße,9,98090,133909,DIS Dresden International School gemeinnützige GmbH,01067,Dresden,Annenstr.,9,1.0,1,1,1
697,Dresden International School,01067,Dresden,Annenstraße,9,98090,143899,Japanische Ergänzungsschule in Dresden e.V.,01067,Dresden,Annenstr.,9,1.0,1,1,1
1628,"Dr. med. Antje Heilmann, Dr. med. Helge Tomczak",01067,Dresden,Friedrichstraße,38,128064,144377,Kinderzentrum Dresden-Friedrichstadt GmbH,01067,Dresden,Friedrichstr.,38,1.0,1,1,2
1630,Praxis für Kinderkardiologie,01067,Dresden,Friedrichstraße,38,90608,144377,Kinderzentrum Dresden-Friedrichstadt GmbH,01067,Dresden,Friedrichstr.,38,1.0,1,1,2
1644,Praxis für Humangenetik,01067,Dresden,Friedrichstraße,38-40,94753,136812,Medizinisches Versorgungszentrum Mitteldeutscher Praxisverbund Humangenetik GmbH,01067,Dresden,Friedrichstr.,38-40,1.0,1,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1215878,Apotheke zur Grünen Schlange,90768,Fürth,Kapellenplatz,1,15293,336076,Apotheke Zur grünen Schlange Johannes Kimberger e. K.,90768,Fürth,Kapellenplatz,1,1.0,1,1,2202
1216593,Frosch-Apotheke,90768,Fürth,Vacher Straße,462,17145,334619,Frosch - Apotheke e. K.,90768,Fürth,Vacher Str.,462,1.0,1,1,2203
1217995,SF Gebäudereinigung,95030,Hof,Hofer Straße,21,27772,332126,SF Gebäudereinigung GmbH,95030,Hof,Hofer Str.,21,1.0,1,1,2204
1218666,CITYCOM GmbH,97239,Aub,Industriestraße,5-9,16236,294641,CITYCOM GmbH,97239,Aub,Industriestr.,5-9,1.0,1,1,2205


In [40]:
ver2 = matched_lower.merge(
    mm, how="outer", on=["COMPANYNAME_OSM", "COMPANYNAME_HUG"], indicator=True
)
ver2["_merge"] = ver2["_merge"].replace(
    {"right_only": "MatrixMatcher", "both": "both", "left_only": "Fine-Tuning"}
)
ver2.columns

Index(['COMPANYNAME_OSM', 'POSTALCODE_OSM_x', 'CITY_OSM_x', 'STREET_OSM_x',
       'HSNR_OSM_x', 'INDEX_SOURCE_OSM', 'INDEX_SOURCE_HUG', 'COMPANYNAME_HUG',
       'POSTALCODE_HUG_x', 'CITY_HUG_x', 'STREET_HUG_x', 'HSNR_HUG_x',
       'HSNR_SIM', 'PREDICTIONS_STREET', 'PREDICTIONS_COMPANY', 'MATCH_ID',
       'CITY_OSM_y', 'POSTALCODE_OSM_y', 'STREET_OSM_y', 'HSNR_OSM_y', 'STATE',
       'match_ID', 'STREET_HUG_y', 'POSTALCODE_HUG_y', 'CITY_HUG_y',
       'HSNR_HUG_y', 'COMPANYNAME_worddiff', 'STREET_worddiff_locality',
       'HSNR_housenumber', 'POSTALCODE_identity', 'CITY_worddiff_locality',
       'CASE', '_merge'],
      dtype='object')

In [71]:
ver2["_merge"].value_counts()

_merge
Fine-Tuning      1846
both             1488
MatrixMatcher     656
Name: count, dtype: int64

In [42]:
ver2[ver2["_merge"] == "MatrixMatcher"][["COMPANYNAME_worddiff"]].value_counts()

COMPANYNAME_worddiff
1.000000                381
0.884587                103
0.952712                 64
0.974476                 17
0.844024                 17
                       ... 
0.895507                  1
0.937855                  1
0.899207                  1
0.803997                  1
0.926443                  1
Name: count, Length: 36, dtype: int64

In [44]:
ver2[ver2["_merge"] == "Fine-Tuning"][["COMPANYNAME_OSM", "COMPANYNAME_HUG"]].sample(10)

,COMPANYNAME_OSM,COMPANYNAME_HUG
2532,Opel Mätschke,Karl Mätschke Vermögensverwaltung GmbH & Co.KG
1762,Jung & Bosch,Jung & Bosch Verwaltungs-GmbH
3271,Staroscik & Staroscik,37Creative Group UG (haftungsbeschränkt)
3467,TÜV Hanse Service-Center Hamburg-Mitte,TÜV Hanse GmbH TÜV SÜD Gruppe
2599,Paulusschule,Förderverein der evangelischen Grundschule Paulus Schule unterm Kirchturm/Dependance Bugenhagenschule e.V.
3962,steinhoff,steinhoff einrichten + wohnen GmbH
3135,Smartphone Care,Kompanie der Kreativen gUG (haftungsbeschränkt)
76,Addon Personal & Lösungen,add-on Forschungs- & Entwicklungsgesellschaft mbH
1389,Grundschule Kaltenweide,Schulverein der GS Kaltenweide e.V.
526,Bibliothek Weinböhla,Zentralgasthof Weinböhla GmbH


In [19]:
ver2["LEV_DIST"] = ver2.apply(
    lambda row: levenstein_rario(row["COMPANYNAME_OSM"], row["COMPANYNAME_HUG"]), axis=1
)
ver2["_merge"].value_counts()

_merge
left_only     1846
both          1488
right_only     656
Name: count, dtype: int64

In [32]:
ver2[ver2["_merge"] == "right_only"]

,COMPANYNAME_OSM,POSTALCODE_OSM_x,CITY_OSM_x,STREET_OSM_x,HSNR_OSM_x,INDEX_SOURCE_OSM,INDEX_SOURCE_HUG,COMPANYNAME_HUG,POSTALCODE_HUG_x,CITY_HUG_x,...,CITY_HUG_y,HSNR_HUG_y,COMPANYNAME_worddiff,STREET_worddiff_locality,HSNR_housenumber,POSTALCODE_identity,CITY_worddiff_locality,CASE,_merge,LEV_DIST
40,AKQA,NaN,NaN,NaN,NaN,NaN,NaN,AKQA GmbH,NaN,NaN,...,Berlin,4,1.000000,1.0,1.0,1.0,1.0,case_0,right_only,0.615385
43,APPLICATIO Training & Management GmbH,NaN,NaN,NaN,NaN,NaN,NaN,Applicatio Training und Management GmbH,NaN,NaN,...,Hamburg,8,0.974476,1.0,1.0,1.0,1.0,case_0,right_only,0.710526
45,ARTvivendi Wohnungs- und Grundstücksverwaltungsgesellschaft mbH,NaN,NaN,NaN,NaN,NaN,NaN,Art vivendi Wohnungs- und Grundstücksverwaltungsgesellschaft mbH,NaN,NaN,...,Berlin,4,0.973121,1.0,1.0,1.0,1.0,case_0,right_only,0.960630
67,Abacco,NaN,NaN,NaN,NaN,NaN,NaN,ABACCO-Hotels GmbH,NaN,NaN,...,Korntal-Münchingen,121,0.884587,1.0,1.0,1.0,1.0,case_0,right_only,0.166667
68,Abacco Hotels GmbH,NaN,NaN,NaN,NaN,NaN,NaN,ABACCO-Hotels GmbH,NaN,NaN,...,Korntal-Münchingen,121,0.884587,1.0,1.0,1.0,1.0,case_0,right_only,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3949,psd Bank RheinNeckarSaar eG,NaN,NaN,NaN,NaN,NaN,NaN,PSD Bank RheinNeckarSaar eG,NaN,NaN,...,Stuttgart,37-39,1.000000,1.0,1.0,1.0,1.0,case_0,right_only,0.888889
3951,rising systems AG,NaN,NaN,NaN,NaN,NaN,NaN,rising systems AG,NaN,NaN,...,Düsseldorf,99,1.000000,1.0,1.0,1.0,1.0,case_0,right_only,1.000000
3968,uhrzeit.org GmbH,NaN,NaN,NaN,NaN,NaN,NaN,Uhrzeit.org GmbH,NaN,NaN,...,Hamburg,7,1.000000,1.0,1.0,1.0,1.0,case_0,right_only,0.937500
3972,ver.di Service GmbH,NaN,NaN,NaN,NaN,NaN,NaN,ver.di Service GmbH,NaN,NaN,...,Berlin,1,1.000000,1.0,1.0,1.0,1.0,case_0,right_only,1.000000


In [20]:
nur_ft = ver2[ver2["_merge"] == "left_only"][
    [
        "MATCH_ID",
        "COMPANYNAME_OSM",
        "COMPANYNAME_HUG",
        "POSTALCODE_OSM_x",
        "CITY_OSM_x",
        "STREET_OSM_x",
        "HSNR_OSM_x",
        "INDEX_SOURCE_OSM",
        "INDEX_SOURCE_HUG",
        "POSTALCODE_HUG_x",
        "CITY_HUG_x",
        "STREET_HUG_x",
        "HSNR_HUG_x",
        "HSNR_SIM",
        "PREDICTIONS_STREET",
        "PREDICTIONS_COMPANY",
        "LEV_DIST",
    ]
]


nur_ft.sort_values("MATCH_ID")

,MATCH_ID,COMPANYNAME_OSM,COMPANYNAME_HUG,POSTALCODE_OSM_x,CITY_OSM_x,STREET_OSM_x,HSNR_OSM_x,INDEX_SOURCE_OSM,INDEX_SOURCE_HUG,POSTALCODE_HUG_x,CITY_HUG_x,STREET_HUG_x,HSNR_HUG_x,HSNR_SIM,PREDICTIONS_STREET,PREDICTIONS_COMPANY,LEV_DIST
928,1.0,Dresden International School,Japanische Ergänzungsschule in Dresden e.V.,01067,Dresden,Annenstraße,9,98090,143899,01067,Dresden,Annenstr.,9,1.0,1.0,1.0,0.253521
897,2.0,"Dr. med. Antje Heilmann, Dr. med. Helge Tomczak",Kinderzentrum Dresden-Friedrichstadt GmbH,01067,Dresden,Friedrichstraße,38,128064,144377,01067,Dresden,Friedrichstr.,38,1.0,1.0,1.0,0.318182
2673,2.0,Praxis für Kinderkardiologie,Kinderzentrum Dresden-Friedrichstadt GmbH,01067,Dresden,Friedrichstraße,38,90608,144377,01067,Dresden,Friedrichstr.,38,1.0,1.0,1.0,0.289855
2672,3.0,Praxis für Humangenetik,Medizinisches Versorgungszentrum Mitteldeutscher Praxisverbund Humangenetik GmbH,01067,Dresden,Friedrichstraße,38-40,94753,136812,01067,Dresden,Friedrichstr.,38-40,1.0,1.0,1.0,0.388350
3578,5.0,Volkswagenzentrum Dresden,Volkswagen Zentrum Dresden GmbH & Co. KG,01067,Dresden,Hamburger Straße,24,89824,143810,01067,Dresden,Hamburger Str.,24,1.0,1.0,1.0,0.738462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3280,2198.0,Stehmann Mode,Stehmann Handels KG,90766,Fürth,Hansastraße,22,14654,341098,90766,Fürth,Hansastr.,22,1.0,1.0,1.0,0.687500
1154,2199.0,Flamme Möbel Fürth GmbH & Co. KG,Flamme Möbel Fürth Verwaltungs-GmbH,90766,Fürth,Hardstraße,80,110797,338413,90766,Fürth,Hardstr.,80,1.0,1.0,1.0,0.686567
1153,2199.0,Flamme Möbel Fürth,Flamme Möbel Fürth Verwaltungs-GmbH,90766,Fürth,Hardstraße,80,25436,338413,90766,Fürth,Hardstr.,80,1.0,1.0,1.0,0.679245
1938,2200.0,Klinikum Fürth,Palliativ-Care Team Fürth gemeinnützige GmbH,90766,Fürth,Jakob-Henle-Straße,1,27671,332472,90766,Fürth,Jakob-Henle-Str.,1,1.0,1.0,1.0,0.344828


In [58]:
nur_ft["LEV_DIST"].astype(str).str[:3].value_counts()

LEV_DIST
0.4    362
0.3    349
0.5    296
0.2    267
0.6    201
0.7     91
0.1     87
0.8     75
1.0     68
0.9     43
0.0      7
Name: count, dtype: int64

In [65]:
nur_ft[["COMPANYNAME_OSM", "COMPANYNAME_HUG", "LEV_DIST"]].sample(10)

,COMPANYNAME_OSM,COMPANYNAME_HUG,LEV_DIST
2520,Oberschule Sabel Freital gAG,Schulförderverein Sabel Freital e.V.,0.625000
587,Brinkmann Eduard Gewürzmühle GmbH,Eduard Brinkmann Verwaltungs GmbH,0.545455
1148,FiZ-Media,FiZ Verwaltungsgesellschaft mbH,0.250000
485,Bauentwurf Pirna,Bauentwurf Pirna GmbH Planungsbüro Vetter - von Berg,0.470588
3899,junited Autoglas,junited AUTOGLAS Service GmbH,0.400000
2941,Rusche-Apotheke,Hauptstadt Ruschestraße 103 GmbH,0.340426
905,"Dr. med. Michaela Wilke, Dr. med. Robert Wilke",Blagovita GmbH,0.200000
2750,REWE,REWE Markt Maik Böttger oHG,0.258065
2359,Michelle Records,VEB Michelle GmbH,0.545455
3925,netCrew GmbH,netCrew Beratungsgesellschaft für Computernetzwerke und Internet mbH,0.275000


In [70]:
nur_ft[nur_ft["LEV_DIST"] < 0.15][
    ["COMPANYNAME_OSM", "COMPANYNAME_HUG", "LEV_DIST"]
].sample(10)

,COMPANYNAME_OSM,COMPANYNAME_HUG,LEV_DIST
2953,S.I.S. Gruppe,Lekkertrade GmbH,0.137931
1323,George P. Johnson,glazy GmbH,0.148148
1982,Kosmetik,UVITY GmbH,0.111111
1962,Kokosh,Schulz & Friemuth Grundstücksverwaltung KG,0.083333
1829,Kees,Kees Fahrschulen Verwaltungsgesellschaft UG (haftungsbeschränkt),0.117647
2205,MVZ Prof. Neuhann,Orthopädisch-Anästhesiologisches-Neurochirurgisches Versorgungszentrum im Helios (MVZ) Partnerschaftsgesellschaft Dr. med.,0.143885
1268,GEH8,GEH8 KUNST RAUM ATELIERS gemeinnützige UG (haftungsbeschränkt),0.121212
2017,Krüger,Feinkost Böhm GmbH,0.083333
466,BW-Bank,SI-BW Geschäftsführungsgesellschaft mbH,0.130435
2519,Obermeyer,Tessloff Verlag Ragnar Tessloff GmbH & Co KG,0.113208


In [26]:
nur_ft[nur_ft["LEV_DIST"] >= 0.5][
    ["COMPANYNAME_OSM", "COMPANYNAME_HUG", "LEV_DIST", "MATCH_ID"]
].sort_values("MATCH_ID").head(20)

,COMPANYNAME_OSM,COMPANYNAME_HUG,LEV_DIST,MATCH_ID
3576,Volkswagenzentrum Dresden,Auto Zentrum Dresden GmbH & Co. KG,0.508475,5.0
3578,Volkswagenzentrum Dresden,Volkswagen Zentrum Dresden GmbH & Co. KG,0.738462,5.0
3575,Volkswagen Zentrum Dresden GmbH & Co. KG,Volkswagen Zentrum Dresden GmbH & Co. KG,1.000000,5.0
3574,Volkswagen Zentrum Dresden GmbH & Co. KG,Auto Zentrum Dresden GmbH & Co. KG,0.837838,5.0
3509,VOL-Stahl GmbH,VOL Immobilien GmbH,0.545455,8.0
3980,winkler.digital GmbH,winkler.digital UG (haftungsbeschränkt),0.610169,9.0
1693,Impact Hub,Impact Dresden GmbH,0.551724,23.0
267,Augustus-Apotheke,Augustus Apotheke Dr. Arnold Hertzsch e.Kfm.,0.524590,27.0
1443,HSS GmbH & Co. KG,HSS GmbH & Co. KG,1.000000,33.0
1256,Förderverein Sächsischer Schülervertretungen e.V.,Förderverein Sächsischer Schülervertretungen e.V.,1.000000,50.0


In [ ]:
nur_ft.sort_values("MATCH_ID")[10:30]

In [ ]:
ver2[ver2["_merge"] == "right_only"][
    [
        "COMPANYNAME_OSM_y",
        "COMPANYNAME_HUG_y",
        "CITY_OSM_y",
        "POSTALCODE_OSM_y",
        "STREET_OSM_y",
        "HSNR_OSM_y",
        "STATE",
        "match_ID",
        "STREET_HUG_y",
        "POSTALCODE_HUG_y",
        "CITY_HUG_y",
        "HSNR_HUG_y",
        "COMPANYNAME_worddiff",
        "STREET_worddiff_locality",
        "HSNR_housenumber",
        "POSTALCODE_identity",
        "CITY_worddiff_locality",
        "CASE",
    ]
]

## Feature extraction 

In [ ]:
output_model_path_street = "C:/Users/sina_/Bachelor_Arbeit/Src/Feature_Extractions/Street_Model/Full/street_classifier.joblib"

output_model_path_company = "C:/Users/sina_/Bachelor_Arbeit/Src/Feature_Extractions/Company_Model/Full/company_classifier.joblib"
matched_fe = matcher(
    df_osm,
    df_hugging,
    method="feature_extraction",
    street_model_path=output_model_path_street,
    company_model_path=output_model_path_company,
)
matched_fe